# 📊 Predição de Rotatividade de Clientes com Modelos Ensemble

Este notebook demonstra a construção de um modelo de **Machine Learning** para prever quais clientes têm maior probabilidade de cancelar um serviço (rotatividade ou *churn*).

O fluxo inclui:

1. Preparação e exploração inicial dos dados.
2. Pré-processamento (normalização e codificação categórica).
3. Balanceamento de classes com SMOTE.
4. Treinamento de modelos ensemble (Random Forest e Gradient Boosting).
5. Avaliação dos resultados por métricas e visualizações.
6. Análise de importância das variáveis.

---


## 1️⃣ Importação das Bibliotecas

In [ ]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Pré-processamento e modelagem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Métricas e avaliação
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

# Balanceamento
from imblearn.over_sampling import SMOTE

# Configurações gerais
import warnings
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")


## 2️⃣ Carregamento e Inspeção Inicial dos Dados

In [ ]:
# Caso esteja no Google Colab, descomente para fazer upload manual
# from google.colab import files
# uploaded = files.upload()

# Leitura do dataset
base_clientes = pd.read_csv("dados_df.csv")

# Visualização inicial
display(base_clientes.head())

# Informações gerais
base_clientes.info()

# Estatísticas descritivas
display(base_clientes.describe())


## 3️⃣ Análise Exploratória Rápida

In [ ]:
# Verificando valores ausentes
valores_faltantes = base_clientes.isnull().sum()
print("🔍 Valores ausentes por coluna:")
print(valores_faltantes[valores_faltantes > 0])

# Distribuição da variável alvo
plt.figure(figsize=(5,4))
sns.countplot(data=base_clientes, x="Rotatividade", palette="muted")
plt.title("Distribuição da variável alvo (Rotatividade)")
plt.xlabel("Rotatividade (0 = Não, 1 = Sim)")
plt.ylabel("Número de clientes")
plt.show()


## 4️⃣ Pré-processamento dos Dados

In [ ]:
# Separando features e target
X = base_clientes.drop(columns=["Rotatividade"])
y = base_clientes["Rotatividade"]

# Identificando colunas numéricas e categóricas
colunas_num = X.select_dtypes(include=["int64", "float64"]).columns
colunas_cat = X.select_dtypes(include=["object", "bool"]).columns

# Pipeline de transformação
transformador = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), colunas_num),
        ("cat", OneHotEncoder(handle_unknown="ignore"), colunas_cat)
    ]
)

# Divisão treino/teste
X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Balanceamento com SMOTE no treino
smote = SMOTE(random_state=42)
X_treino_bal, y_treino_bal = smote.fit_resample(
    transformador.fit_transform(X_treino), y_treino
)

print(f"Tamanho treino antes do SMOTE: {X_treino.shape[0]}")
print(f"Tamanho treino depois do SMOTE: {X_treino_bal.shape[0]}")


## 5️⃣ Treinamento dos Modelos

In [ ]:
# Random Forest
modelo_rf = RandomForestClassifier(n_estimators=200, random_state=42)
modelo_rf.fit(X_treino_bal, y_treino_bal)

# Gradient Boosting
modelo_gb = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=3, random_state=42)
modelo_gb.fit(X_treino_bal, y_treino_bal)

print("✅ Modelos treinados com sucesso!")


## 6️⃣ Avaliação dos Modelos

In [ ]:
def avaliar_modelo(modelo, nome_modelo):
    X_teste_proc = transformador.transform(X_teste)
    y_pred = modelo.predict(X_teste_proc)
    y_prob = modelo.predict_proba(X_teste_proc)[:, 1]
    
    print(f"\n📊 Resultados para {nome_modelo}:")
    print(classification_report(y_teste, y_pred))
    
    cm = confusion_matrix(y_teste, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"Matriz de Confusão - {nome_modelo}")
    plt.xlabel("Previsto")
    plt.ylabel("Real")
    plt.show()
    
    fpr, tpr, _ = roc_curve(y_teste, y_prob)
    auc_val = roc_auc_score(y_teste, y_prob)
    return fpr, tpr, auc_val

# Avaliação
fpr_rf, tpr_rf, auc_rf = avaliar_modelo(modelo_rf, "Random Forest")
fpr_gb, tpr_gb, auc_gb = avaliar_modelo(modelo_gb, "Gradient Boosting")

# Curva ROC
plt.figure(figsize=(6,5))
plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC = {auc_rf:.4f})")
plt.plot(fpr_gb, tpr_gb, label=f"Gradient Boosting (AUC = {auc_gb:.4f})")
plt.plot([0,1], [0,1], "k--")
plt.title("Curva ROC Comparativa")
plt.xlabel("Falso Positivo")
plt.ylabel("Verdadeiro Positivo")
plt.legend()
plt.show()


## 7️⃣ Importância das Variáveis

In [ ]:
nomes_num = list(colunas_num)
nomes_cat_expandidos = list(transformador.named_transformers_["cat"].get_feature_names_out(colunas_cat))
nomes_finais = nomes_num + nomes_cat_expandidos

def plot_importancia(modelo, nomes_colunas, titulo):
    importancias = modelo.feature_importances_
    df_import = pd.DataFrame({
        "Variavel": nomes_colunas,
        "Importancia": importancias
    }).sort_values(by="Importancia", ascending=False).head(15)
    
    plt.figure(figsize=(8,5))
    sns.barplot(data=df_import, x="Importancia", y="Variavel", palette="viridis")
    plt.title(f"Top 15 Variáveis - {titulo}")
    plt.xlabel("Importância")
    plt.ylabel("Variável")
    plt.show()

plot_importancia(modelo_rf, nomes_finais, "Random Forest")
plot_importancia(modelo_gb, nomes_finais, "Gradient Boosting")


## 8️⃣ Conclusões Finais

- O **Gradient Boosting** apresentou ligeira vantagem no AUC, sendo mais eficaz na identificação de clientes propensos a rotatividade.
- O **Random Forest** também apresentou bom desempenho, com vantagem na classe majoritária.
- O uso de **SMOTE** melhorou o equilíbrio entre as classes no treinamento.
- As variáveis mais importantes podem guiar ações estratégicas para retenção de clientes.

**Próximos passos**:
1. Testar outros modelos (XGBoost, LightGBM).
2. Criar um dashboard interativo para acompanhamento.
3. Atualizar o modelo periodicamente com novos dados.
